In [1]:
import numpy as np
import pandas as pd # for loading data

In [123]:
cd ..

/home/nicktehrany/Documents/Uni/3rd Year/P4/Machine Learning/Project


In [124]:
# Importing our own Functions
from util.helperfunctions import one_hot, one_hot_genres, cnn_clean_text, one_hot_reverse

In [125]:
cd src

/home/nicktehrany/Documents/Uni/3rd Year/P4/Machine Learning/Project/src


In [5]:
train_df = pd.read_csv("train.csv")

In [6]:
lyrics, max_len, word_list = cnn_clean_text(train_df['Lyrics'], 1)

# writing cleaned text back to df
index = 0
for text in lyrics:
    train_df['Lyrics'][index] = text
    index+=1
del lyrics

In [7]:
words = []
words.append('<PAD>')
for word in word_list:
    words.append(word)

In [8]:
train_lyrics = np.zeros(shape=(len(train_df), max_len))

row_index = 0
for row in train_df['Lyrics']:
    single_words = row.split()
    col_index = 0
    for word in single_words:
        train_lyrics[row_index][col_index] = words.index(word)
        col_index+=1
    row_index+=1

In [9]:
print(train_lyrics.shape)

(10098, 1525)


In [14]:
train_genres = one_hot_genres(train_df['Genre'])

In [15]:
val_df = pd.read_csv("validation.csv")
lyrics, dummy_len, dummy_list = cnn_clean_text(val_df['Lyrics'], 0)

# writing cleaned text back to df
index = 0
for text in lyrics:
    val_df['Lyrics'][index] = text
    index+=1
del lyrics

In [16]:
val_genres = one_hot_genres(val_df['Genre'])

In [17]:
val_lyrics = np.zeros(shape=(len(val_df), max_len))

row_index = 0
for row in val_df['Lyrics']:
    single_words = row.split()
    col_index = 0
    for word in single_words:
        if word in word_list:
            val_lyrics[row_index][col_index] = words.index(word)
        col_index+=1
    row_index+=1

In [18]:
test_df = pd.read_csv("test.csv")
lyrics, dummy_len, dummy_list = cnn_clean_text(test_df['Lyrics'], 0)

# writing cleaned text back to df
index = 0
for text in lyrics:
    test_df['Lyrics'][index] = text
    index+=1
del lyrics

In [19]:
test_genres = one_hot_genres(test_df['Genre'])

In [20]:
test_lyrics = np.zeros(shape=(len(test_df), max_len))

row_index = 0
for row in test_df['Lyrics']:
    single_words = row.split()
    col_index = 0
    for word in single_words:
        if word in word_list:
            test_lyrics[row_index][col_index] = words.index(word)
        col_index+=1
    row_index+=1

In [178]:
word2vec = {}
with open('w2v_sg_model.word2vec', encoding='UTF-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec

In [179]:
EMBEDDING_DIM = 100
embedding_matrix = np.zeros((len(words), EMBEDDING_DIM))
index = 0
for word in words:
    embedding_vector = word2vec.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[index] = embedding_vector
    index+=1


In [180]:
from keras.layers import Embedding

embedding_layer = Embedding(len(words),
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=max_len,
                            trainable=False)

In [181]:
from keras.models import Model
from keras.layers import Dense, Input, Activation, Conv1D, MaxPooling1D, Dropout, Flatten

sequence_input = Input(shape=(max_len,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
conv1= Conv1D(128, 2, activation='relu')(embedded_sequences)
pool1 = MaxPooling1D(2)(conv1)
conv2 = Conv1D(128, 2, activation='relu')(pool1)
pool2 = MaxPooling1D(2)(conv2)
conv3 = Conv1D(128, 2, activation='relu')(pool2)
pool3 = MaxPooling1D(2)(conv3)
flatten = Flatten()(pool3)
last = Dense(28, activation='relu')(flatten)
final = Dense(8, activation='softmax')(last)
model = Model(sequence_input, final)

model.summary()

Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_24 (InputLayer)        (None, 1525)              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 1525, 100)         3426100   
_________________________________________________________________
conv1d_43 (Conv1D)           (None, 1524, 128)         25728     
_________________________________________________________________
max_pooling1d_43 (MaxPooling (None, 762, 128)          0         
_________________________________________________________________
conv1d_44 (Conv1D)           (None, 761, 128)          32896     
_________________________________________________________________
max_pooling1d_44 (MaxPooling (None, 380, 128)          0         
_________________________________________________________________
conv1d_45 (Conv1D)           (None, 379, 128)          328

In [182]:
from keras.optimizers import Adam

optimizer = Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['acc'])

model.fit(train_lyrics, train_genres, epochs=5, shuffle=True, batch_size=16, validation_data=(val_lyrics, val_genres))

Train on 10098 samples, validate on 2142 samples
Epoch 1/5
10098/10098 [==============================] - 40s 4ms/step - loss: 1.8361 - acc: 0.2654 - val_loss: 1.7082 - val_acc: 0.3366
Epoch 2/5
10098/10098 [==============================] - 38s 4ms/step - loss: 1.5749 - acc: 0.3924 - val_loss: 1.5420 - val_acc: 0.4118
Epoch 3/5
10098/10098 [==============================] - 38s 4ms/step - loss: 1.4749 - acc: 0.4341 - val_loss: 1.5225 - val_acc: 0.4225
Epoch 4/5
10098/10098 [==============================] - 38s 4ms/step - loss: 1.3987 - acc: 0.4708 - val_loss: 1.5347 - val_acc: 0.4220
Epoch 5/5
10098/10098 [==============================] - 38s 4ms/step - loss: 1.3155 - acc: 0.5015 - val_loss: 1.5492 - val_acc: 0.4239


In [183]:
results = model.evaluate(test_lyrics, test_genres, batch_size=16)
print("Accuracy: ", results[1])

2160/2160 [==============================] - 2s 1ms/step
Accuracy:  0.4314814805984497


In [177]:
from sklearn.metrics import confusion_matrix

y_pred=model.predict(test_lyrics)
cm=confusion_matrix(one_hot_reverse(test_genres),y_pred)
print(cm)

ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

genre_list = ['Pop','Rock', 'Hip-Hop/Rap', 'Country', 'R&B/Soul', 'Metal', 'Alternative/Indie', 'Folk']

df_cm = pd.DataFrame(cm, index = genre_list, columns = genre_list)
print(df_cm)
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True, linewidth=0.5, fmt='g', cmap="BuPu")
b, t = plt.ylim() # discover the values for bottom and top
b += 0.5 # Add 0.5 to the bottom
t -= 0.5 # Subtract 0.5 from the top
plt.ylim(b, t) # update the ylim(bottom, top) values